<a href="https://colab.research.google.com/github/NihPat95/Instacart-Market-Analysis/blob/master/Instacart_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install efficient_apriori

In [0]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display
import time
from efficient_apriori import apriori
import sys
import random 
colors = sns.color_palette()

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
path = '/content/drive/My Drive/Instacart/'
orders = pd.read_pickle(path + 'customers_orders10000.pkl')
products = pd.read_csv(path + 'products.csv')
productIdMap = products.set_index(['product_id'])['product_name'].to_dict()

In [0]:
# Function that returns the size of an object in MB
def size(obj):
    return "{0:.2f} MB".format(sys.getsizeof(obj) / (1000 * 1000))

def itemsets_id_to_name(itemsets):
  itemsets_names = {}
  for k, v in itemsets.items():
    itemsets_names[k] = {}
    for k2, v2 in v.items():
      itemsets_names[k][tuple([productIdMap[i] for i in k2])] = v2
  return itemsets_names

def rules_id_to_name(rules):
  rules_name = [Rule(tuple([productIdMap[i] for i in rule.lhs]), 
                   tuple([productIdMap[i] for i in rule.rhs]),
                   rule.count_full, rule.count_lhs, 
                   rule.count_rhs, rule.num_transactions) for rule in rules]
  return rules_name

In [0]:
def compute_prediction(h1, h2, rule):
  predicted = [[i for i in rule.rhs] for rule in rule if set(rule.lhs).issubset(h1)]
  predicted = [item for sublist in predicted for item in sublist]
  predicted = set(predicted)
  return predicted

def compute_recall(rules, X_test):
  test = X_test['items'].map(lambda x: random.sample(x, len(x)))
  recall = []
  for b in test:
    h1, h2 = set(b[0:int(len(b)/2)]), set(b[int(len(b)/2):])
    predicted = compute_prediction(h1, h2, rules)
    recall.append(len(predicted.intersection(h2))/len(h2))  
  return sum(recall) / len(recall) * 100 

In [0]:
from sklearn.model_selection import KFold
cv = KFold(n_splits=25, random_state=42, shuffle=False)

In [31]:
scores = []
for train_index, test_index in cv.split(orders['items']):
  X_train, X_test = orders.iloc[train_index], orders.iloc[test_index]
  _, rules = apriori(X_train['items'], min_support=0.01,  min_confidence=0.01)
  scores.append(compute_recall(rules, X_test))

CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 7.63 µs


In [32]:
scores

[7.317073170731707,
 5.894308943089431,
 5.691056910569105,
 8.130081300813007,
 9.552845528455284,
 8.94308943089431,
 7.520325203252034,
 9.146341463414634,
 7.317073170731707,
 9.552845528455284,
 6.504065040650407,
 7.723577235772358,
 7.113821138211382,
 8.536585365853659,
 7.317073170731707,
 7.755102040816326,
 6.73469387755102,
 13.673469387755102,
 7.346938775510205,
 6.938775510204081,
 4.8979591836734695,
 11.224489795918368,
 3.2653061224489797,
 6.530612244897959,
 8.36734693877551]